## Sveska za importovanje i pravljenje jobova za CAS-VIT mrežu

CAS-ViT se zasniva na Visual Transformer arhitekturi, više informacija na linku

https://paperswithcode.com/paper/cas-vit-convolutional-additive-self-attention

Ova sveska uz prateće fajlove automatski skida weightove za CAS-ViT model zadate veličine, i posle taj model šalje na AIHUB za testiranje.

Za sada sam pokrenuo samo xs model, koji se izvršava super brzo i ima manje parametara od MobileNetV2, doduše mnogo više layera koji su raspoređeni na više COMPUTE unita, što je koliko ja razumem dobro.

Ne znam da li je dobro da modele čuvamo na GIT-u (Drakula neka kaže kako bi to trebalo), pa sam sve fajlove koje se generišu stavio u gitignore.


In [ ]:
import gdown
import torch
import rcvit
import torchviz
import torchsummary
import numpy as np
import qai_hub as hub
import requests
import input_getter
import make_jobs

In [ ]:
# Model selection (options: "xs", "s", "m", "t")

modelID = "xs"

driveFileIDs = dict()

driveFileIDs["xs"] = "16wKcwF6QMW5w_lyPYnDKjMNuoxQDfrLK"
driveFileIDs["s"]  = "1facFRq8s8oelYUtK1fj3fcfdoWoKDBQQ"
driveFileIDs["m"]  = "13sQpSEf0h_uuh0jRy9V0yIW6ZsbDpVGy"
driveFileIDs["t"]  = "1NqoIUPbwBC91RTjTUvubAbOfGqo1VYT0"

networks = dict()
networks["xs"] = rcvit.rcvit_xs
networks["s"]  = rcvit.rcvit_s
networks["m"]  = rcvit.rcvit_m
networks["t"]  = rcvit.rcvit_t

file_id = driveFileIDs[modelID]

modelName = "CASVIT_{}".format(modelID)

file_path = modelName + ".pth"

#gdown.download(f"https://drive.google.com/uc?id={file_id}", file_path, quiet=False)

print("Model downloaded successfully.")

In [ ]:
net: rcvit.RCViT = networks[modelID]()
print(net)

In [ ]:
torchsummary.summary(net, (3, 224, 224))

In [ ]:
# Loading pre-trained weights

weightDict = torch.load(file_path, map_location=torch.device("cpu"))
net.load_state_dict(weightDict["model"])

In [ ]:
# Kada se uradi ovi jobovi na linkovima se moze naci vizuelizacija mreze i informacije za dalju analizu

net.eval()

input = input_getter.random_input_getter()

c, p, i = make_jobs.compile_profile_inference(net, input)
